# 04 - Estimation globale du paramètre xi

Dans ce notebook, nous allons construire un estimateur global pour le paramètre xi des GEV aux stations. Nous allons pour cela devoir construire une fonction à $2N+1$ paramètres, où $N$ est le nombre de stations : en effet, il faut estimer en même temps les paramètres $\mu_{i}$, $\phi_{i}$ et enfin $\xi$.

D'après notre précédente étude sur les corrélations des stations d'une même cellule, nous en sommes venus à la conclusion qu'il fallait mieux les concaténer, pour une même cellule. C'est donc ce que nous allons faire.

In [1]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim

using IDF

import Plots

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

DURATION = "24 h"

"24 h"

In [3]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


In [4]:
load_station("8100512")

,Year,Duration,Pcp
,Int64,Cat…,Float64
1,1959,5 min,9.7
2,1960,5 min,5.3
3,1961,5 min,7.4
4,1962,5 min,5.6
5,1963,5 min,3.6
6,1964,5 min,3.6
7,1965,5 min,11.7
8,1966,5 min,3.0
9,1967,5 min,7.9


In [ ]:
function f(p::Vector{:Float64}, N::Int64)
    if length(p) != 2*N+1
        error("The vector of parameters is not of the good length")
    end
    
    s=0
    for i in 1:N
        id = station_list[i, :ID]
        s -= logL(pcp(id, DURATION), p[i], p[N+i], p[2*N+1])